<a href="https://colab.research.google.com/github/danie-bit/deep-learning/blob/main/V_54.3_age_gender_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
!kaggle datasets download -d jangedoo/utkface-new


Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [22]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [23]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [24]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [25]:
len(age)

23708

In [26]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [27]:
df.shape

(23708, 3)

In [28]:
df.head()

,age,gender,img
0,18,1,18_1_4_20170103222943622.jpg.chip.jpg
1,1,0,1_0_2_20161219195258747.jpg.chip.jpg
2,33,1,33_1_0_20170103182556882.jpg.chip.jpg
3,80,1,80_1_0_20170110141723527.jpg.chip.jpg
4,12,0,12_0_0_20170110224833703.jpg.chip.jpg


In [29]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [30]:
train_df.shape

(20000, 3)

In [31]:
test_df.shape

(3708, 3)

In [32]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [33]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [34]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [35]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [36]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [37]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [38]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [44]:
# model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

In [40]:
def multi_output_generator(generator):
    while True:
        images, labels = next(generator)
        # labels is a list: [age_labels, gender_labels]
        # We need to convert it to a dictionary: {'age': age_labels, 'gender': gender_labels}
        yield images, {'age': labels[0], 'gender': labels[1]}

# Create new wrapped generators
wrapped_train_generator = multi_output_generator(train_generator)
wrapped_test_generator = multi_output_generator(test_generator)

print("Custom multi-output generator created for training and testing.")

Custom multi-output generator created for training and testing.


In [43]:
steps_per_epoch = len(train_df) // 32
validation_steps = len(test_df) // 32

model.fit(wrapped_train_generator,
          steps_per_epoch=steps_per_epoch,
          epochs=10,
          validation_data=wrapped_test_generator,
          validation_steps=validation_steps)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 266s 425ms/step - age_loss: 14.8956 - age_mae: 14.8956 - gender_accuracy: 0.5175 - gender_loss: 0.6925 - loss: 83.4578 - val_age_loss: 14.3200 - val_age_mae: 14.3200 - val_gender_accuracy: 0.5193 - val_gender_loss: 0.6924 - val_loss: 82.8698
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 255s 408ms/step - age_loss: 14.7417 - age_mae: 14.7417 - gender_accuracy: 0.5279 - gender_loss: 0.6925 - loss: 83.2950 - val_age_loss: 14.7715 - val_age_mae: 14.7715 - val_gender_accuracy: 0.5220 - val_gender_loss: 0.6923 - val_loss: 83.3082
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 267s 427ms/step - age_loss: 14.9531 - age_mae: 14.9531 - gender_accuracy: 0.5245 - gender_loss: 0.6950 - loss: 83.7588 - val_age_loss: 14.0915 - val_age_mae: 14.0900 - val_gender_accuracy: 0.5204 - val_gender_loss: 0.6924 - val_loss: 82.6352
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 259s 415ms/step - age_loss: 14.6149 - age_mae: 14.6149 - gender_accuracy: 0.5210 - gender_loss: 0.6942 - loss: 83.